In [1]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import yahoofinancials
import pandas as pd
import numpy as np
import utility as u


In [2]:
log_transform = False
add_regressor = False
country = "JP"


In [3]:
start, end = '2019-01-01', '2020-04-03'
train_start_date, train_end_date = start, '2020-03-01'
test_start_date, test_end_date = '2020-03-02', end 

stock = 'TSLA'
df = yf.download(stock, start=start, end=end, progress=False)
df["y"] = df['High']+df['Low']
df.index = [i.date() for i in df.index]


In [4]:
df.reset_index(inplace=True)
df.rename(columns={"index":"ds"}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'])
df = df[['ds', 'y']]


In [5]:
if log_transform:
    df['y'] = np.log(df['y'])
    

In [6]:
train_df = df[(df.ds>=train_start_date) & (df.ds<=train_end_date)]
test_df = df[(df.ds>=test_start_date) & (df.ds<=test_end_date)]

In [7]:
# Get the holidays
holidays_df = u.add_holidays(train_df, train_start_date, train_end_date, country)


In [8]:
# Prophet model and predicted values
prophet_model, forecast = u.generate_model(train_df, 
                                           holidays_df, 
                                           periods = 60, 
                                           add_regressor = add_regressor)

if log_transform:
    forecast['yhat'] = np.exp(forecast['yhat'])
    forecast['yhat_lower'] = np.exp(forecast['yhat_lower'])
    forecast['yhat_upper'] = np.exp(forecast['yhat_upper'])
    train_df['y'] = np.exp(train_df['y'])
    

In [9]:
# Compare actuals vs predictions 
# in sample fit
compare_df = u.compare_pred(train_df, forecast, plot=True)


In [10]:
# View seasonality components in the model
# Use prophet_model.plot_components(forecast) to use Prophet's built-in charts
u.plot_components(prophet_model, forecast)

In [11]:
test_set_size = int((pd.Timestamp(train_end_date) - pd.Timestamp(train_start_date)).days * 0.75)
test_set_size_str = f'{test_set_size} days'


In [12]:
# View the cross validation and error plots
# out of sample fit
df_p = u.cross_validate(prophet_model, initial=test_set_size_str)